In [2]:
import pandas as pd
import os
import numpy as np
import re

In [3]:
#!pip install ace_tools

In [4]:
base_dir = os.path.dirname(os.getcwd())
data_path = os.path.join(base_dir, 'data', 'veridion_product_deduplication_challenge.snappy.parquet')

#load parquet file
df = pd.read_parquet(data_path)

#preview
df.head()

,unspsc,root_domain,page_url,product_title,product_summary,product_name,product_identifier,brand,intended_industries,applicability,...,form,size,color,purity,energy_efficiency,pressure_rating,power_rating,quality_standards_and_certifications,miscellaneous_features,description
0,Sewing and stitchery and weaving equipment and...,studio-atcoat.com,https://studio-atcoat.com/1372696759/?idx=510,Glimakra Warping Board (8m),The Glimakra Warping Board is designed for use...,Warping Board,[],None,[Textile],[use with floor looms],...,[],"[{'dimension': 'Length', 'qualitative': False,...",[],[],None,[],[],[],[],"The ""Warping Board"" is designed for use with f..."
1,Electric alternating current AC motors,worm-gears.net,https://worm-gears.net/tag/worm-gear-box/,NMRV Worm Gearbox Motor,The NMRV Worm Gearbox Motor is a high-efficien...,Worm Gearbox Motor,[],None,[Industrial],[industrial applications],...,[],[],"[{'original': 'Blue', 'simple': 'Blue'}, {'ori...",[],None,[],"[{'qualitative': False, 'type': 'min', 'unit':...",[],"[Omnibearing installation, High radiation effi...","The ""Worm Gearbox Motor"" is a high-efficiency ..."
2,Vehicle trim and exterior covering,customcarcoverco.com,https://customcarcoverco.com/collections/vendo...,Nissan R33 GTR Car Cover,A custom car cover designed for the Nissan R33...,Car Cover,[],None,[Automotive],[protecting vehicles from the elements],...,[],[],[],[],None,[],[],[],"[Personalization with custom brand logos, grap...","The ""Car Cover"" is a custom-designed cover tai..."
3,Pipe connectors,plumbmaster.com,https://www.plumbmaster.com/search?q=wolverine...,Flexible Fittings,"Flexible fittings for plumbing applications, a...",Flexible Fittings,[],None,[Plumbing],[plumbing installations],...,[],[],[],[],None,[],[],[],"[allows for movement, flexibility in installat...","""Flexible Fittings"" are designed for plumbing ..."
4,Doors,sogno.in,http://www.sogno.in/product-detail-CST-HGD-331...,CST-HGD-33103 Hinged Closet Door,The CST-HGD-33103 Hinged Closet Door is a meti...,Hinged Closet Door,[],CST,"[Home Appliances, Construction]",[Closet Storage],...,[],[],[],[],None,[],[],[],"[Italian craftsmanship, German engineering, Sm...","The ""Hinged Closet Door"" is a storage solution..."


## 1.  Explore Data



In [5]:
df.shape

(21946, 31)

In [6]:
df.columns

Index(['unspsc', 'root_domain', 'page_url', 'product_title', 'product_summary',
       'product_name', 'product_identifier', 'brand', 'intended_industries',
       'applicability', 'eco_friendly', 'ethical_and_sustainability_practices',
       'production_capacity', 'price', 'materials', 'ingredients',
       'manufacturing_countries', 'manufacturing_year', 'manufacturing_type',
       'customization', 'packaging_type', 'form', 'size', 'color', 'purity',
       'energy_efficiency', 'pressure_rating', 'power_rating',
       'quality_standards_and_certifications', 'miscellaneous_features',
       'description'],
      dtype='object')

In [7]:
# Missing value detector- not only NaN's in the dataset but also empty lists, dicts strings etc.
def is_effectively_missing(x):
    if isinstance(x, (list, np.ndarray)):
        return len(x) == 0
    elif pd.isnull(x):
        return True
    elif isinstance(x, str) and x.strip() == '':
        return True
    return False

# Apply the function to detect missing values
missing_counts = df.applymap(is_effectively_missing).sum()
missing_percent = (missing_counts / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing %': missing_percent.round(2)
}).sort_values(by='Missing Count', ascending=False)


display(missing_df)

C:\Users\asus\AppData\Local\Temp\ipykernel_9936\635080358.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  missing_counts = df.applymap(is_effectively_missing).sum()


,Missing Count,Missing %
production_capacity,21911,99.84
purity,21773,99.21
energy_efficiency,21769,99.19
ethical_and_sustainability_practices,21586,98.36
form,21447,97.73
power_rating,21379,97.42
pressure_rating,21370,97.38
ingredients,21337,97.23
manufacturing_type,21280,96.97
eco_friendly,20977,95.58


In [8]:
#reduce noise- decide threshold value 91.46% missing values to drop completely 
#product identifier 91.46 as threshold as given by name might still be useful for enrichment
columns_to_drop = [
    'production_capacity', 'purity', 'energy_efficiency', 'ethical_and_sustainability_practices',
    'form', 'power_rating', 'pressure_rating', 'ingredients', 'manufacturing_type',
    'eco_friendly', 'quality_standards_and_certifications', 'packaging_type',
    'manufacturing_countries'
]
df = df.drop(columns=columns_to_drop)

In [9]:
#keep ones from 67 to 91.46 just for enrichment as they still might be informative post-clustering, not for core deduplication logic
df.shape

(21946, 18)

## 2.  Preprocessing:

I will address the scalability issues in the modular code, this notebook is just a representation for the workflow of the project


In [10]:
df.head(n=5)  

,unspsc,root_domain,page_url,product_title,product_summary,product_name,product_identifier,brand,intended_industries,applicability,price,materials,manufacturing_year,customization,size,color,miscellaneous_features,description
0,Sewing and stitchery and weaving equipment and...,studio-atcoat.com,https://studio-atcoat.com/1372696759/?idx=510,Glimakra Warping Board (8m),The Glimakra Warping Board is designed for use...,Warping Board,[],None,[Textile],[use with floor looms],"[{'amount': 159000.0, 'currency': 'KRW', 'type...",[],-1,[],"[{'dimension': 'Length', 'qualitative': False,...",[],[],"The ""Warping Board"" is designed for use with f..."
1,Electric alternating current AC motors,worm-gears.net,https://worm-gears.net/tag/worm-gear-box/,NMRV Worm Gearbox Motor,The NMRV Worm Gearbox Motor is a high-efficien...,Worm Gearbox Motor,[],None,[Industrial],[industrial applications],[],[],-1,[],[],"[{'original': 'Blue', 'simple': 'Blue'}, {'ori...","[Omnibearing installation, High radiation effi...","The ""Worm Gearbox Motor"" is a high-efficiency ..."
2,Vehicle trim and exterior covering,customcarcoverco.com,https://customcarcoverco.com/collections/vendo...,Nissan R33 GTR Car Cover,A custom car cover designed for the Nissan R33...,Car Cover,[],None,[Automotive],[protecting vehicles from the elements],[],[],-1,"[Customizable with custom brand logos, graphic...",[],[],"[Personalization with custom brand logos, grap...","The ""Car Cover"" is a custom-designed cover tai..."
3,Pipe connectors,plumbmaster.com,https://www.plumbmaster.com/search?q=wolverine...,Flexible Fittings,"Flexible fittings for plumbing applications, a...",Flexible Fittings,[],None,[Plumbing],[plumbing installations],[],[],-1,[],[],[],"[allows for movement, flexibility in installat...","""Flexible Fittings"" are designed for plumbing ..."
4,Doors,sogno.in,http://www.sogno.in/product-detail-CST-HGD-331...,CST-HGD-33103 Hinged Closet Door,The CST-HGD-33103 Hinged Closet Door is a meti...,Hinged Closet Door,[],CST,"[Home Appliances, Construction]",[Closet Storage],[],[],-1,[],[],[],"[Italian craftsmanship, German engineering, Sm...","The ""Hinged Closet Door"" is a storage solution..."


In [11]:
from collections import Counter

for col in df.columns:
    sample = df[col].dropna().head(100)  # sample first 100 non-null entries
    types = [type(x).__name__ for x in sample]
    type_counts = Counter(types)
    print(f"{col}: {dict(type_counts)}")

unspsc: {'str': 100}
root_domain: {'str': 100}
page_url: {'str': 100}
product_title: {'str': 100}
product_summary: {'str': 100}
product_name: {'str': 100}
product_identifier: {'ndarray': 100}
brand: {'str': 100}
intended_industries: {'ndarray': 100}
applicability: {'ndarray': 100}
price: {'ndarray': 100}
materials: {'ndarray': 100}
manufacturing_year: {'int': 100}
customization: {'ndarray': 100}
size: {'ndarray': 100}
color: {'ndarray': 100}
miscellaneous_features: {'ndarray': 100}
description: {'str': 100}


In [12]:
#First- text normalization.
#for str apply to all except punctuation for root domain+ url
import re

def normalize_text(text, remove_punct=True):
    if isinstance(text, str):
        text = text.lower()
        if remove_punct:
            text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
        text = text.replace('\n', ' ')
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ""

text_columns = ['product_title', 'product_summary', 'product_name','unspsc','description']

for col in text_columns:
    df[col] = df[col].apply(lambda x: normalize_text(x, remove_punct=True))
    
def normalize_product_identifier(val):
    if isinstance(val, (list, np.ndarray)):
        if len(val) == 0:
            return np.nan
        cleaned = [normalize_text(x) for x in val if isinstance(x, str) and x.strip()]
        return ', '.join(sorted(set(cleaned))) if cleaned else np.nan
    return np.nan

df['product_identifier'] = df['product_identifier'].apply(normalize_product_identifier)
df['brand'] = df['brand'].apply(lambda x: normalize_text(x, remove_punct=True) if isinstance(x, str) else np.nan)

def normalize_array_field(arr):
    if isinstance(arr, (list, np.ndarray)):
        cleaned = [normalize_text(x) for x in arr if isinstance(x, str) and x.strip()]
        return ', '.join(sorted(set(cleaned))) if cleaned else np.nan
    return np.nan

#If we are dealing WITH LARGE DATA. care with apply here, I will change the approach in the modular implementation
df['intended_industries'] = df['intended_industries'].apply(normalize_array_field)
df['applicability'] = df['applicability'].apply(normalize_array_field)

def extract_price_amount(x):
    if isinstance(x, np.ndarray) and len(x) > 0 and isinstance(x[0], dict):
        return x[0].get('amount', np.nan)
    return np.nan

def extract_price_currency(x):
    if isinstance(x, np.ndarray) and len(x) > 0 and isinstance(x[0], dict):
        return normalize_text(x[0].get('currency', ''), remove_punct=False)
    return np.nan

def extract_price_type(x):
    if isinstance(x, np.ndarray) and len(x) > 0 and isinstance(x[0], dict):
        return normalize_text(x[0].get('type', ''), remove_punct=True)
    return np.nan

# Apply extraction
df['price_amount'] = df['price'].apply(extract_price_amount)
df['price_currency'] = df['price'].apply(extract_price_currency)
df['price_type'] = df['price'].apply(extract_price_type)


df.drop(columns=['price'], inplace=True)


df['price_amount'] = pd.to_numeric(df['price_amount'], errors='coerce').astype('float64').round(2)

def normalize_materials(arr):
    if isinstance(arr, (list, np.ndarray)):
        cleaned = [normalize_text(x) for x in arr if isinstance(x, str) and x.strip()]
        return ', '.join(sorted(set(cleaned))) if cleaned else np.nan
    return np.nan

df['materials'] = df['materials'].apply(normalize_materials)
df.drop(columns=['manufacturing_year'], inplace=True)

df.head(n=7)

,unspsc,root_domain,page_url,product_title,product_summary,product_name,product_identifier,brand,intended_industries,applicability,materials,customization,size,color,miscellaneous_features,description,price_amount,price_currency,price_type
0,sewing and stitchery and weaving equipment and...,studio-atcoat.com,https://studio-atcoat.com/1372696759/?idx=510,glimakra warping board 8m,the glimakra warping board is designed for use...,warping board,NaN,NaN,textile,use with floor looms,NaN,[],"[{'dimension': 'Length', 'qualitative': False,...",[],[],the warping board is designed for use with flo...,159000.0,krw,exact
1,electric alternating current ac motors,worm-gears.net,https://worm-gears.net/tag/worm-gear-box/,nmrv worm gearbox motor,the nmrv worm gearbox motor is a highefficienc...,worm gearbox motor,NaN,NaN,industrial,industrial applications,NaN,[],[],"[{'original': 'Blue', 'simple': 'Blue'}, {'ori...","[Omnibearing installation, High radiation effi...",the worm gearbox motor is a highefficiency gea...,NaN,NaN,NaN
2,vehicle trim and exterior covering,customcarcoverco.com,https://customcarcoverco.com/collections/vendo...,nissan r33 gtr car cover,a custom car cover designed for the nissan r33...,car cover,NaN,NaN,automotive,protecting vehicles from the elements,NaN,"[Customizable with custom brand logos, graphic...",[],[],"[Personalization with custom brand logos, grap...",the car cover is a customdesigned cover tailor...,NaN,NaN,NaN
3,pipe connectors,plumbmaster.com,https://www.plumbmaster.com/search?q=wolverine...,flexible fittings,flexible fittings for plumbing applications al...,flexible fittings,NaN,NaN,plumbing,plumbing installations,NaN,[],[],[],"[allows for movement, flexibility in installat...",flexible fittings are designed for plumbing ap...,NaN,NaN,NaN
4,doors,sogno.in,http://www.sogno.in/product-detail-CST-HGD-331...,csthgd33103 hinged closet door,the csthgd33103 hinged closet door is a meticu...,hinged closet door,NaN,cst,"construction, home appliances",closet storage,NaN,[],[],[],"[Italian craftsmanship, German engineering, Sm...",the hinged closet door is a storage solution c...,NaN,NaN,NaN
5,faucets or taps,plumbmaster.com,https://www.plumbmaster.com/search?q=wolverine...,deep faucets,faucets with a deep design providing a secure ...,deep faucets,NaN,NaN,plumbing,NaN,NaN,[],[],[],"[deep design, secure and stable connection]",deep faucets are designed with a deep design t...,NaN,NaN,NaN
6,dispensing tools,advancedpressuresystems.ca,https://advancedpressuresystems.ca/collections...,10k dry shutoff gun handle assembly,the 10k dry shutoff gun handle assembly is a c...,dry shutoff gun handle assembly,NaN,NaN,manufacturing,highpressure water blasting operations,NaN,[],[],[],[],the dry shutoff gun handle assembly is a compo...,NaN,NaN,NaN


In [13]:
def normalize_customization(arr):
    if isinstance(arr, (list, np.ndarray)):
        cleaned = [normalize_text(x) for x in arr if isinstance(x, str) and x.strip()]
        return ', '.join(sorted(set(cleaned))) if cleaned else np.nan
    return np.nan

df['customization'] = df['customization'].apply(normalize_customization)

def normalize_misc_features(arr):
    if isinstance(arr, (list, np.ndarray)):
        cleaned = [normalize_text(x) for x in arr if isinstance(x, str) and x.strip()]
        return ', '.join(sorted(set(cleaned))) if cleaned else np.nan
    return np.nan

df['miscellaneous_features'] = df['miscellaneous_features'].apply(normalize_misc_features)

In [14]:
df.head(n=7)

,unspsc,root_domain,page_url,product_title,product_summary,product_name,product_identifier,brand,intended_industries,applicability,materials,customization,size,color,miscellaneous_features,description,price_amount,price_currency,price_type
0,sewing and stitchery and weaving equipment and...,studio-atcoat.com,https://studio-atcoat.com/1372696759/?idx=510,glimakra warping board 8m,the glimakra warping board is designed for use...,warping board,NaN,NaN,textile,use with floor looms,NaN,NaN,"[{'dimension': 'Length', 'qualitative': False,...",[],NaN,the warping board is designed for use with flo...,159000.0,krw,exact
1,electric alternating current ac motors,worm-gears.net,https://worm-gears.net/tag/worm-gear-box/,nmrv worm gearbox motor,the nmrv worm gearbox motor is a highefficienc...,worm gearbox motor,NaN,NaN,industrial,industrial applications,NaN,NaN,[],"[{'original': 'Blue', 'simple': 'Blue'}, {'ori...","good service life, high radiation efficiency, ...",the worm gearbox motor is a highefficiency gea...,NaN,NaN,NaN
2,vehicle trim and exterior covering,customcarcoverco.com,https://customcarcoverco.com/collections/vendo...,nissan r33 gtr car cover,a custom car cover designed for the nissan r33...,car cover,NaN,NaN,automotive,protecting vehicles from the elements,NaN,customizable with custom brand logos graphics ...,[],[],personalization with custom brand logos graphi...,the car cover is a customdesigned cover tailor...,NaN,NaN,NaN
3,pipe connectors,plumbmaster.com,https://www.plumbmaster.com/search?q=wolverine...,flexible fittings,flexible fittings for plumbing applications al...,flexible fittings,NaN,NaN,plumbing,plumbing installations,NaN,NaN,[],[],"allows for movement, flexibility in installation",flexible fittings are designed for plumbing ap...,NaN,NaN,NaN
4,doors,sogno.in,http://www.sogno.in/product-detail-CST-HGD-331...,csthgd33103 hinged closet door,the csthgd33103 hinged closet door is a meticu...,hinged closet door,NaN,cst,"construction, home appliances",closet storage,NaN,NaN,[],[],"german engineering, italian craftsmanship, smo...",the hinged closet door is a storage solution c...,NaN,NaN,NaN
5,faucets or taps,plumbmaster.com,https://www.plumbmaster.com/search?q=wolverine...,deep faucets,faucets with a deep design providing a secure ...,deep faucets,NaN,NaN,plumbing,NaN,NaN,NaN,[],[],"deep design, secure and stable connection",deep faucets are designed with a deep design t...,NaN,NaN,NaN
6,dispensing tools,advancedpressuresystems.ca,https://advancedpressuresystems.ca/collections...,10k dry shutoff gun handle assembly,the 10k dry shutoff gun handle assembly is a c...,dry shutoff gun handle assembly,NaN,NaN,manufacturing,highpressure water blasting operations,NaN,NaN,[],[],NaN,the dry shutoff gun handle assembly is a compo...,NaN,NaN,NaN


In [15]:
# unique_colors = df['color'].dropna().apply(str).unique().tolist()
# unique_colors.sort()
# for color in unique_colors:
#     print(color)
# #they seem to have the same format. original -simple.
def normalize_color(arr):
    if isinstance(arr, (list, np.ndarray)):
        values = []
        for entry in arr:
            if isinstance(entry, dict):
                val = entry.get('simple') or entry.get('original')
                if val:
                    values.append(normalize_text(val))
        return ', '.join(sorted(set(values))) if values else np.nan
    return np.nan

df['color'] = df['color'].apply(normalize_color)

In [16]:
unique_sizes = df['size'].dropna().apply(str).unique().tolist()
unique_sizes.sort()
for size in unique_sizes:
    print(size)

[]
[{'dimension': 'Area', 'qualitative': False, 'type': 'exact', 'unit': 'ft2', 'value': '1500'}]
[{'dimension': 'Area', 'qualitative': False, 'type': 'exact', 'unit': 'ft2', 'value': '700'}]
[{'dimension': 'Area', 'qualitative': False, 'type': 'exact', 'unit': 'in2', 'value': '254'}]
[{'dimension': 'Area', 'qualitative': False, 'type': 'exact', 'unit': 'in2', 'value': '4'}]
[{'dimension': 'Area', 'qualitative': False, 'type': 'exact', 'unit': 'm2', 'value': '1'}]
[{'dimension': 'Area', 'qualitative': False, 'type': 'exact', 'unit': 'm2', 'value': '1.76'}]
[{'dimension': 'Area', 'qualitative': False, 'type': 'exact', 'unit': 'm2', 'value': '100'}]
[{'dimension': 'Area', 'qualitative': False, 'type': 'exact', 'unit': 'm2', 'value': '18'}]
[{'dimension': 'Area', 'qualitative': False, 'type': 'exact', 'unit': 'm2', 'value': '500'}]
[{'dimension': 'Area', 'qualitative': False, 'type': 'max', 'unit': 'ft2', 'value': '1800'}
 {'dimension': 'Area', 'qualitative': False, 'type': 'min', 'unit':

In [17]:
def normalize_size_field(arr):
    if isinstance(arr, (list, np.ndarray)) and len(arr) > 0:
        parts = []
        for entry in arr:
            if isinstance(entry, dict):
                dim = normalize_text(entry.get('dimension', ''), remove_punct=False)
                val = normalize_text(entry.get('value', ''), remove_punct=False)
                unit = normalize_text(entry.get('unit', ''), remove_punct=False)
                typ = normalize_text(entry.get('type', ''), remove_punct=False)

                if dim and val and unit:
                    label = f"{dim}: {typ} {val} {unit}" if typ else f"{dim}: {val} {unit}"
                    parts.append(label.strip())
        return ', '.join(parts) if parts else np.nan
    return np.nan

df['size'] = df['size'].apply(normalize_size_field)

In [18]:
df

,unspsc,root_domain,page_url,product_title,product_summary,product_name,product_identifier,brand,intended_industries,applicability,materials,customization,size,color,miscellaneous_features,description,price_amount,price_currency,price_type
0,sewing and stitchery and weaving equipment and...,studio-atcoat.com,https://studio-atcoat.com/1372696759/?idx=510,glimakra warping board 8m,the glimakra warping board is designed for use...,warping board,NaN,NaN,textile,use with floor looms,NaN,NaN,length: exact 8 m,NaN,NaN,the warping board is designed for use with flo...,159000.0,krw,exact
1,electric alternating current ac motors,worm-gears.net,https://worm-gears.net/tag/worm-gear-box/,nmrv worm gearbox motor,the nmrv worm gearbox motor is a highefficienc...,worm gearbox motor,NaN,NaN,industrial,industrial applications,NaN,NaN,NaN,"blue, gray","good service life, high radiation efficiency, ...",the worm gearbox motor is a highefficiency gea...,NaN,NaN,NaN
2,vehicle trim and exterior covering,customcarcoverco.com,https://customcarcoverco.com/collections/vendo...,nissan r33 gtr car cover,a custom car cover designed for the nissan r33...,car cover,NaN,NaN,automotive,protecting vehicles from the elements,NaN,customizable with custom brand logos graphics ...,NaN,NaN,personalization with custom brand logos graphi...,the car cover is a customdesigned cover tailor...,NaN,NaN,NaN
3,pipe connectors,plumbmaster.com,https://www.plumbmaster.com/search?q=wolverine...,flexible fittings,flexible fittings for plumbing applications al...,flexible fittings,NaN,NaN,plumbing,plumbing installations,NaN,NaN,NaN,NaN,"allows for movement, flexibility in installation",flexible fittings are designed for plumbing ap...,NaN,NaN,NaN
4,doors,sogno.in,http://www.sogno.in/product-detail-CST-HGD-331...,csthgd33103 hinged closet door,the csthgd33103 hinged closet door is a meticu...,hinged closet door,NaN,cst,"construction, home appliances",closet storage,NaN,NaN,NaN,NaN,"german engineering, italian craftsmanship, smo...",the hinged closet door is a storage solution c...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21941,other,dsbridal.com,https://www.dsbridal.com/index.php/sale/veils....,1516 accessories,accessories designed for sweet 1516 available ...,accessories,NaN,NaN,retail,accessories for sweet 1516,NaN,NaN,diameter: exact 15 in,NaN,part of the sale collection,accessories are designed for use with sweet 15...,NaN,NaN,NaN
21942,processed and synthetic rubber,50735-in.all.biz,https://50735-in.all.biz/group-goods,general mechanical rubber goods,a category of rubber goods designed for genera...,rubber goods,NaN,NaN,manufacturing,NaN,rubber,NaN,NaN,NaN,NaN,rubber goods are designed for general mechanic...,NaN,NaN,NaN
21943,fresh cut rose bouquets,lilyofthevalley.uk,https://www.lilyofthevalley.uk/product/luxurio...,luxurious rose garden,the luxurious rose garden is a stunning floral...,floral arrangement,NaN,lily of the valley florist,"gifts, retail","decorative purposes, gifting",NaN,NaN,"width: exact 250 cm, height: exact 250 cm",NaN,product images available in various resolutions,the floral arrangement offered by lily of the ...,45.0,gbp,min
21944,vision correction or cosmetic eyewear and rela...,getcontactlensesonline.com.au,https://getcontactlensesonline.com.au/brand/al...,dailies aquacomfort plus multifocal 30 pack,a pack of 30 dailies aquacomfort plus multifoc...,multifocal contact lenses,NaN,dailies,healthcare,vision correction,NaN,NaN,NaN,NaN,NaN,multifocal contact lenses are designed for dai...,82.0,usd,exact


In [23]:
import os

os.makedirs("../Data", exist_ok=True)
df.to_csv("../Data/cleaned_products.csv", index=False)

#for local use

In [24]:
import os

os.makedirs("../Data", exist_ok=True)
df.to_parquet("../Data/cleaned_products.parquet", index=False)

#for cluster use